# Singleton
A component which is instantiated only once.
However, it is difficult to be tested -> Use mockup.

## Motivation
- For some components it only makes sense to have one in the system
  - Database repository
  - Object Factory

- E.g., the initializer call is expensive
  - We only do it once
  - We provide everyone with the same instance

- Want to prevent anyone creating additional copies
- Need to take cae of lazy instantiation

## Python Implementation
- Custom Allocator
- Decorator
- Metaclass*
- Monostate

In [ ]:
# Singleton Allocator

import random

class Database:
    __instance = None

    def __init__(self):
        self.id = random.randint(1, 101)
        print("Generated an id of ", self.id)        

    def __new__(cls, *args, **kwargs):
        if not cls.__instance:
            cls.__instance = super(Database, cls).__new__(cls, *args, **kwargs)
        return cls.__instance


database = Database()

d1 = Database()  # id is overwritten
d2 = Database()  # id is overwritten

print(d1.id, d2.id)
print(d1 == d2)
print(database == d1)
print(d1 is d2)
print()
print(dir(d1))  # id is in the attribute
print(d1.__dict__)
print(getattr(d1, 'id'))

In [10]:
# Singleton Decorator

def singleton(class_name):
    instances = {}  # closure, defining

    def get_instance(*args, **kwargs):
        if class_name not in instances:
            print(f'creating singleton instance({class_name}), {instances.items()}')
            instances[class_name] = class_name(*args, **kwargs)
        return instances[class_name]

    return get_instance

# def singleton(class_name):
#     instance = None

#     def get_instance(*args, **kwargs):
#         nonlocal instance
#         if instance is None:            
#             instance = class_name(*args, **kwargs)
#             print(f'creating singleton instance({class_name})')
#         return instance

#     return get_instance


@singleton
class Database:
    def __init__(self):
        print('Loading database')

@singleton
class Network:
    def __init__(self):
        print('start networking...')

print("-----------------------")
d1 = Database()
d2 = Database()
print(d1 == d2)
print(d1 is d2)

n1 = Network()
n2 = Network()
print(n1 == n2)
print(n1 is n2)
print(d1 is n1)

-----------------------
creating singleton instance(<class '__main__.Database'>), dict_items([])
Loading database
True
True
creating singleton instance(<class '__main__.Network'>), dict_items([])
start networking...
True
True
False


In [1]:
# Singleton Metaclass
class Singleton(type):
    """ Metaclass that creates a Singleton base type when called. """

    # __instances = {}

    # def __call__(cls, *args, **kwargs):
    #     if cls not in cls.__instances:
    #         cls.__instances[cls] = super().__call__(*args, **kwargs)
    #         print(f'creating singleton instance({cls}  {len(cls.__instances)})')
    #     return cls.__instances[cls]

    __instances = None

    def __call__(cls, *args, **kwargs):
        if cls.__instances is None:
            cls.__instances = super().__call__(*args, **kwargs)
            print(f'creating singleton instance({cls})')
        return cls.__instances

    


class Database(metaclass=Singleton):
    def __init__(self):
        print("Loading database")
    def __str__(self):
        return "DataBase"

class Network(metaclass=Singleton):
    def __init__(self):
        print("Loading network")
    def __str__(self):
        return "Network"


d1 = Database()
d2 = Database()
print(d1 == d2)
print(d1 is d2)

n1 = Network()
n2 = Network()
print(n1 == n2)
print(n1 is n2)
print(d1 is n1)


Loading database
creating singleton instance(<class '__main__.Database'>)
True
True
Loading network
creating singleton instance(<class '__main__.Network'>)
True
True
False


In [20]:
# monostate #1
# monostate is a conceptual singleton. all members are static :)

class CEO:
    __shared_state = {
        'name': 'Steve',
        'age': 55
    }

    def __init__(self):
        self.__dict__ = self.__shared_state  # move class variable values to object variables

    def __str__(self):
        return f'{self.name} is {self.age} years old'
    
    def __eq__(self, rhs):
        return self.name == rhs.name and self.age == rhs.age

ceo1 = CEO()
print(ceo1)
ceo1.age = 66
print(ceo1)

ceo2 = CEO()
ceo2.age = 77
print(ceo1)
print(ceo2)
ceo2.name = 'Tim'

ceo3 = CEO()
print(ceo3)
print(ceo1 == ceo2)  # can be false without __eq__
print(ceo1 is ceo2)  # False!
print(id(ceo1), id(ceo2), id(ceo3))

Steve is 55 years old
Steve is 66 years old
Steve is 77 years old
Steve is 77 years old
Tim is 77 years old
True
False
2176097390304 2176097388960 2176097390496


In [46]:
# monostate #2 with Inheritance

class Monostate:
    _shared_state = {}

    def __new__(cls, *args, **kwargs):
        #obj = super(Monostate, cls).__new__(cls, *args, **kwargs)  # allocation on every __new__
        obj = super().__new__(cls, *args, **kwargs)  # allocation on every __new__
        obj.__dict__ = cls._shared_state
        return obj

class CFO(Monostate):
    def __init__(self):
        self.name = ''
        self.money_managed = 0

    def __str__(self):
        return f'{self.name} manages ${self.money_managed}bn'
    
    def __eq__(self, rhs):
        return self.name == rhs.name and self.money_managed == rhs.money_managed

cfo1 = CFO()
cfo1.name = 'Sheryl'
cfo1.money_managed = 1

print(cfo1)

cfo2 = CFO()
cfo2.name = 'Ruth'
cfo2.money_managed = 10
print(cfo1, cfo2, sep='\n')

print(cfo1 == cfo2)  # can be false without __eq__
print(cfo1 is cfo2)

Sheryl manages $1bn
Ruth manages $10bn
Ruth manages $10bn
True
False
